# Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from ast import Import
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Input,Model
from tensorflow.keras.datasets import mnist,cifar10
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalMaxPooling2D,AveragePooling2D,UpSampling2D,Conv2DTranspose,GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense,ZeroPadding2D,Con
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.run_functions_eagerly(True)

#  Generator

In [ ]:
# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=64).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

# Model

In [ ]:

input = Input(shape=(32,32,3))

x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(input)
x=Activation('elu')(x)
x=BatchNormalization()(x)
x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(x)
x=Activation('elu')(x)
x=BatchNormalization()(x)
x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(x)
x=Dropout(0.5)(x)

x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(x)
x=Activation('elu')(x)
x=BatchNormalization()(x)
x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(x)
x=Activation('elu')(x)
x=BatchNormalization()(x)
x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(x)
x=Dropout(0.5)(x)

x=Conv2D(192, (3, 3), padding = 'same', kernel_initializer="he_normal")(x)
x=Activation('elu')(x)

x=Conv2D(192, (1,1), padding = 'same', kernel_initializer="he_normal")(x)
x=Activation('elu')(x)

x=Conv2D(10, (1,1), padding = 'same', kernel_initializer="he_normal")(x)


x =GlobalAveragePooling2D()(x)
output=Activation('softmax')(x)



# Training Functions

In [ ]:
save_callback = keras.callbacks.ModelCheckpoint(
    'chekpoint/',
    save_weights_only=True,
    monitor='accuracy',
    save_best_only=False,
)

def scheduler(epoch, lr):
  if epoch < 2:
    return lr
  else:
     return lr*0.99

lr_Sccheduler = keras.callbacks.LearningRateScheduler (scheduler,verbose=1)


class CustomCallback (keras.callbacks.Callback):
  def on_epoch_end (self, epoch, logs=None):
    #print(logs.keys())
    if logs.get("accuracy") > 0.99:
      print("Accuracy over 90%, qutting training")
      #self.model.stop_training = True

mycallback = CustomCallback()

In [ ]:
# Instantiate an optimizer to train the model.
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

model = keras.Model(inputs=input, outputs=output)

#import datetime
#https://www.tensorflow.org/tensorboard/get_started
train_writer = tf.summary.create_file_writer("logs/train/")
test_writer = tf.summary.create_file_writer("logs/test/")
train_step = test_step = 0



# Proposed Scale Up Gradient Function

In [ ]:
@tf.function
def gradi(gradients):

        for i in range(len(gradients)):

            mean=0
            a=0
            b=0
            c=0
         # CHOOOSIG LAYER OE KEREL TO MIDIFY
            if gradients[i].shape == [3,3,3,32] :
               if i==0:
                  gradient_new = gradients
                  gradient_trasposed=gradients
                  gradient_new1=gradients
                  boolean_mask=gradients
                  gradient_new2=gradients
                  mean = tf.math.reduce_mean(gradients[i], keepdims=True)
                  maskv=[]
                  for e in range (3):
                        for r in range (32):
                            for h in range (3):
                                 for d in range (3):
                                     mean_kernel = tf.math.reduce_mean((tf.Variable(gradients [0][e,:,:,r])), keepdims=False)
                                     std_kernel =tf.math.reduce_std( (tf.Variable(gradients [0][e,:,:,r])), keepdims=False)
                                     t = tf.Variable(gradients [0][e,h,d,r])
                                     if t>1 :
                                        gaus= 2 * tf.math.exp(- (tf.math.pow((t-mean_kernel),2))/2*tf.math.square(std_kernel))
                                     else:
                                        gaus = 0

                                     tf.compat.v1.assign((tf.Variable(gradient_new [0][e,h,d,r])),gaus)

                  gradients[0] = gradient_new [0]
            else:
                 gradients [i] = gradients [i]
        return gradients

# Train and Test Functions

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
        tf.executing_eagerly()
    gradients = tape.gradient(loss_value, model.trainable_weights)
    #Call Proposed Function
    grads = gradi (gradients)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_loss_value = loss_fn(y, val_logits)
    val_acc = val_acc_metric.update_state(y, val_logits)
    return [val_acc,val_loss_value]

# Fitting

In [ ]:
import time
gradients =[]
epochs = 31
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))
    print("Training loss over epoch: %.4f"% (float(loss_value)))

    # Tensorboard Writing Log
    with train_writer.as_default():
      tf.summary.scalar("Adam Kernelized_loss",loss_value,step=epoch)
      tf.summary.scalar ("Adam Kernelized_Accuracy",train_acc, step=epoch)

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_acc,val_loss_value = test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_loss_value = val_loss_value

    # Tensorboard Writing Log
    with test_writer.as_default():
      tf.summary.scalar("Adam Kernelized_loss",val_loss_value,step=epoch)
      tf.summary.scalar ("Adam Kernelized_Accuracy",val_acc, step=epoch)


    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Validation Loss : %.4f" % (float(val_loss_value),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
Training acc over epoch: 0.4205
Training loss over epoch: 1.4679
Validation acc: 0.4807
Validation Loss : 2.2742
Time taken: 107.83s

Start of epoch 1
Training acc over epoch: 0.5917
Training loss over epoch: 1.0891
Validation acc: 0.6286
Validation Loss : 1.7976
Time taken: 103.45s

Start of epoch 2
Training acc over epoch: 0.6638
Training loss over epoch: 0.8896
Validation acc: 0.6473
Validation Loss : 1.9192
Time taken: 106.22s

Start of epoch 3
Training acc over epoch: 0.7104
Training loss over epoch: 0.7118
Validation acc: 0.6967
Validation Loss : 1.4390
Time taken: 103.27s

Start of epoch 4
Training acc over epoch: 0.7489
Training loss over epoch: 0.7111
Validation acc: 0.6985
Validation Loss : 1.3461
Time taken: 102.62s

Start of epoch 5
Training acc over epoch: 0.7742
Training loss over epoch: 0.5948
Validation acc: 0.7164
Validation Loss : 0.9825
Time taken: 102.69s

Start of epoch 6
Training acc over epoch: 0.7998
Training loss over epoch: 0.4461
Validation 

# Tensorbord Visualization

In [ ]:
!tensorboard dev upload --logdir ./"logs" \
--name "My Oprimizer vs Typicall Optimizer" \
--description " Kernelized Adam Optimizer vs Standard Adam Optimizers" \
--one_shot


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

To sign in with the TensorBoard uploader:

1. On your computer or phone, visit:

   https://www.google.com/device

2. Sign in with your Google account, then enter:

   GSYY-VJDS



New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/30APEr2fRQaqhK9koh125g/

[2023-05-23T06:49:05] Started scanning logdir.
[2023-05-23T06:49:06] Total uploaded: 250 scalars, 0 ten

In [ ]:
#Test Loop

for batch_idx, (x_batch, y_batch) in enumerate (ds_test):
  y_pred = model( x_batch)
  acc_metric.update_state (y_batch,y_pred)

train_acc = acc_metric.result()
print(f"Accuracy over Test Set: {train_acc}")
acc_metric.reset_states()


Accuracy over Test Set: 0.9991666674613953
